In [149]:
import pandas as pd
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_absolute_error
import numpy as np
import time

In [150]:
train_df = pd.read_csv('data/train.csv')

In [151]:
#train_df.info()
#train_df[['MSSubClass', 'MSZoning', 'LotArea', 'Utilities', 'Neighborhood',
#'Condition1', 'OverallQual', 'OverallCond', 'YearRemodAdd', 'Exterior1st',
#'ExterCond', 'Foundation', 'HeatingQC', 'Electrical', '1stFlrSF', 'Functional',
#'GarageQual', 'PoolQC', 'Fence', 'MiscFeature', 'MiscVal', 'SaleCondition',
#'YrSold']].isnull().sum()

In [152]:
X = train_df[[
    'MSSubClass',
    'MSZoning',
    'LotArea', 
    #'Utilities', 
    'Neighborhood',
    'Condition1', 
    'OverallQual', 
    'OverallCond', 
    'YearRemodAdd', 
    'Exterior1st',
    'ExterCond', 
    'Foundation', 
    'HeatingQC', 
    'Electrical', 
    #'1stFlrSF',
    'GrLivArea',
    'Functional',
    'GarageQual', 
    'PoolQC', 
    'Fence', 
    'MiscFeature', 
    'MiscVal'
    #'SaleCondition'
    #'YrSold'
]].copy()

imputer = SimpleImputer(strategy='constant', fill_value='NA')
X[[
    'Electrical', 
    'GarageQual', 
    'PoolQC', 
    'Fence', 
    'MiscFeature'
]] = imputer.fit_transform(X[[
    'Electrical', 
    'GarageQual', 
    'PoolQC', 
    'Fence', 
    'MiscFeature'
]])

X['MSSubClass'] = X['MSSubClass'].astype(str)
X = pd.get_dummies(X, drop_first=True)

print("X table size:", X.shape)
print("NA:", X.isnull().sum().sum())

X table size: (1460, 110)
NA: 0


In [153]:
Y = train_df[['SalePrice']].copy()

In [154]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [155]:
housePriceModelXGB = XGBRegressor(
    n_estimators=25000,
    learning_rate=0.001,
    max_depth=5,
    subsample=0.8,
    colsample_bytree=0.6,
    n_jobs=-1,
    random_state=42,
    device='cuda',
    #early_stopping_rounds=100
)

start = time.time()
housePriceModelXGB.fit(X_train, Y_train)
end = time.time()

print(f"\nTraining finished in: {end - start:.3f} seconds")

predictions = housePriceModelXGB.predict(X_test)

mae = mean_absolute_error(Y_test, predictions)

print(f"MedinAverage error {mae:,.2f} dollars") 

feature_importance = pd.DataFrame({
    'feature': X.columns,
    'importance': housePriceModelXGB.feature_importances_
})

feature_importance['base_feature'] = feature_importance['feature'].apply(lambda x: x.split('_')[0])

aggregated_importance = feature_importance.groupby('base_feature')['importance'].sum().sort_values(ascending=False)

print("\nImportance:")
print(aggregated_importance)

/home/renertt/Документы/Machine Learning/housePriceML/venv/lib/python3.10/site-packages/xgboost/training.py:183: UserWarning: [19:36:14] WARNING: /workspace/src/context.cc:49: No visible GPU is found, setting device to CPU.
  bst.update(dtrain, iteration=i, fobj=obj)
/home/renertt/Документы/Machine Learning/housePriceML/venv/lib/python3.10/site-packages/xgboost/training.py:183: UserWarning: [19:36:14] WARNING: /workspace/src/context.cc:203: XGBoost is not compiled with CUDA support.
  bst.update(dtrain, iteration=i, fobj=obj)



Training finished in: 16.150 seconds
MedinAverage error 16,948.00 dollars

Importance:
base_feature
Neighborhood    0.238970
OverallQual     0.162202
Foundation      0.114833
MSSubClass      0.079554
Exterior1st     0.061710
PoolQC          0.050728
MSZoning        0.049768
GarageQual      0.046253
Condition1      0.043824
GrLivArea       0.038085
Functional      0.029001
YearRemodAdd    0.017558
HeatingQC       0.016469
Fence           0.012240
ExterCond       0.010617
LotArea         0.010110
OverallCond     0.006811
Electrical      0.004808
MiscFeature     0.004567
MiscVal         0.001892
Name: importance, dtype: float32


In [156]:
print("Model mistaken by:",round((mae / np.mean(Y_train)) * 100, 2), "%")
#print("Model mistaken by:",((mae / np.mean(Y_train)) * 100), "%")

Model mistaken by: 9.34 %
